# 筆記本 3：繪製事件發生圖表

在這個範例中，我們將學習如何透過 `matplotlib` 和 `pandas` 繪製一個事件發生圖表。本圖表將展示如「落石」和「停電」這類事件在不同日期的分佈，幫助我們分析事件的發生時間趨勢。

### 需求
1. **輸入檔案**：一個 CSV 檔案，檔案中包含事件的日期、類型等資訊。
2. **輸出圖表**：一張展示事件類型和發生日期的散點圖。
3. **畫布設置**：我們將使用顏色來區分不同的事件類型，並將日期作為 X 軸，事件類型作為 Y 軸。

### 主要學習內容
- 如何使用 `pandas` 來處理 CSV 文件。
- 如何使用 `matplotlib` 來繪製包含中文標籤的圖表。
- 如何使用命令行參數來自動設置輸入輸出文件和圖表標題。


In [ ]:
# 安裝必要的 Python 套件
!pip install matplotlib pandas

### 步驟 1: 定義繪製圖表的函數
這個函數負責讀取 CSV 檔案，處理日期和事件類型的數據，並以散點圖的形式顯示事件發生日期。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from matplotlib.dates import DateFormatter, MonthLocator

# 設置中文字型以正確顯示標籤
matplotlib.rc('font', family='Microsoft JhengHei')

def plot_event_occurrences(input_file, output_file, title):
    """
    根據給定的 CSV 檔案繪製事件發生圖表
    """
    # 讀取數據
    data = pd.read_csv(input_file, sep=',', encoding='utf-8')

    # 將日期字符串轉換為日期對象
    data['日期'] = pd.to_datetime(data['日期'])

    # 定義事件類型到英文的映射，以便後續處理
    event_type_mapping = {
        '落石': 'Rockfall',
        '停電': 'Power Outage'
    }

    # 將事件類型轉換為英文，方便在圖上分配顏色
    data['事件類型英文'] = data['事件類型'].map(event_type_mapping)

    # 為不同的事件類型分配不同的顏色
    colors = {
        'Rockfall': 'red',
        'Power Outage': 'blue'
    }

    # 繪製散點圖
    plt.figure(figsize=(12, 6))
    for event_type, color in colors.items():
        subset = data[data['事件類型英文'] == event_type]
        y_position = 1 if event_type == 'Rockfall' else 0.5  # 設定Y軸位置
        plt.scatter(subset['日期'], [y_position] * len(subset),
                    c=color, label=event_type, alpha=0.6, s=50, edgecolor='k')

    # 設置日期格式器和定位器
    plt.gca().xaxis.set_major_locator(MonthLocator())
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y-%m'))

    # 設置標題與Y軸標籤
    plt.title(title, fontsize=13, fontweight='bold')
    plt.ylim(0, 1.5)
    plt.yticks([1, 0.5], ['落石', '停電'], fontsize=13, fontweight='bold')
    plt.grid(True)
    plt.xticks(rotation=45)

    # 調整佈局並保存成圖片
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    plt.show()

# 測試函數
plot_event_occurrences('event_log.txt', 'event_chart.png', '事件發生時間點')

### 步驟 2: 設置命令行參數解析
接下來，我們將定義一個 `ArgumentParser` 類，用來從命令行接收輸入文件、輸出文件和圖表標題等參數。這樣我們之後即可通過命令行方便地進行輸入與輸出文件的指定。


In [ ]:
import argparse

class CustomArgumentParser(argparse.ArgumentParser):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def print_help(self, file=None):
        # 自訂顯示的幫助訊息
        help_message = f"""使用方法: {self.prog} [-h] -I 輸入檔案 -O 輸出圖表 [-T 標題]

繪製事件發生圖表

選項:
  -h, --help            顯示此幫助訊息並退出
  -I INPUT, --input INPUT
                        輸入檔案
  -O OUTPUT, --output OUTPUT
                        輸出圖表
  -T TITLE, --title TITLE
                        圖表標題
        """
        print(help_message, file=file)

# 測試自訂 ArgumentParser
parser = CustomArgumentParser(description="繪製事件發生圖表")
parser.add_argument('-I', '--input', type=str, required=True, help='輸入檔案')
parser.add_argument('-O', '--output', type=str, required=True, help='輸出圖表')
parser.add_argument('-T', '--title', type=str, default="事件發生頻率", help='圖表標題')

### 步驟 3: 主程序入口
最後，我們通過主函數來解析參數，並將輸入參數傳遞給之前的繪圖函數，進行 CSV 資料讀取與散點圖生成。

In [ ]:
import sys

if __name__ == "__main__":
    # 模擬命令列參數
    sys.argv = ['03_visualize_event_trends.py', '-I', 'event_log.txt', '-O', 'event_chart.png', '-T', '事件頻率圖']
    # 設置命令行參數解析
    parser = CustomArgumentParser(description="繪製事件發生圖表")
    parser.add_argument('-I', '--input', type=str,
                        required=True, help='輸入檔案')
    parser.add_argument('-O', '--output', type=str, required=True, help='輸出圖表')
    parser.add_argument('-T', '--title', type=str,
                        default="事件發生頻率", help='圖表標題')

    args = parser.parse_args()

    # 調用繪圖函數
    plot_event_occurrences(args.input, args.output, args.title)


### 結論
在本次教程中，我們學習了如何使用 `matplotlib` 和 `pandas` 繪製事件發生圖表，並用命令行參數來靈活設定輸入和輸出。同時，也介紹了如何使用自訂的字體來處理中文標籤，讓圖表在不同語言環境下都能正確顯示標籤。

如有多次事件或數量較多的情況，可考慮進一步優化圖表，增加互動性或進一步分析事件趨勢。